In [1]:
import tensorflow as tf

from tensorflow.keras import layers, models

In [2]:
import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image

import code

import tensorflow.python.platform

import numpy as np
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
sys.path.append('./')
from helpers.helpers import *

In [5]:
NUM_CHANNELS = 3  # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 20
VALIDATION_SIZE = 5  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16  # 64
NUM_EPOCHS = 100
RESTORE_MODEL = True  # If True, restore existing model instead of training a new one
RECORDING_STEP = 0

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16

In [6]:
data_dir = 'data/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, TRAINING_SIZE)
train_labels = extract_labels(train_labels_filename, TRAINING_SIZE)

Loading data/training/images/satImage_001.png
Loading data/training/images/satImage_002.png
Loading data/training/images/satImage_003.png
Loading data/training/images/satImage_004.png
Loading data/training/images/satImage_005.png
Loading data/training/images/satImage_006.png
Loading data/training/images/satImage_007.png
Loading data/training/images/satImage_008.png
Loading data/training/images/satImage_009.png
Loading data/training/images/satImage_010.png
Loading data/training/images/satImage_011.png
Loading data/training/images/satImage_012.png
Loading data/training/images/satImage_013.png
Loading data/training/images/satImage_014.png
Loading data/training/images/satImage_015.png
Loading data/training/images/satImage_016.png
Loading data/training/images/satImage_017.png
Loading data/training/images/satImage_018.png
Loading data/training/images/satImage_019.png
Loading data/training/images/satImage_020.png
Loading data/training/groundtruth/satImage_001.png
Loading data/training/groundt

In [7]:
num_epochs = NUM_EPOCHS

c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(train_labels)):
    if train_labels[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

print('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(train_labels) if j[0] == 1]
idx1 = [i for i, j in enumerate(train_labels) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print(len(new_indices))
print(train_data.shape)
train_data = train_data[new_indices, :, :, :]
train_labels = train_labels[new_indices]

train_size = train_labels.shape[0]

c0 = 0
c1 = 0
for i in range(len(train_labels)):
    if train_labels[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 9450 c1 = 3050
Balancing training data...
6100
(12500, 16, 16, 3)
Number of data points per class: c0 = 3050 c1 = 3050


In [8]:
train_data.shape

(6100, 16, 16, 3)

In [9]:
if RESTORE_MODEL:
    # It can be used to reconstruct the model identically.
    model = models.load_model("handmade_cnn_save/")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
if not RESTORE_MODEL:
    model = models.Sequential()
    model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(16, 16, 3), padding='same', use_bias=True))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(8, 8, 3), padding='same', use_bias=True))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2))

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 16, 16, 32)        2432      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                65600     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [78]:
if not RESTORE_MODEL: 
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    history = model.fit(train_data, train_labels, epochs=10) #,validation_data=(test_images, test_labels)
    model.save("handmade_cnn_save")

Train on 6100 samples
Epoch 1/10
6100/6100 [==============================] - 3s 542us/sample - loss: 0.0269 - accuracy: 0.9900
Epoch 2/10
6100/6100 [==============================] - 3s 462us/sample - loss: 0.0525 - accuracy: 0.9808
Epoch 3/10
6100/6100 [==============================] - 3s 561us/sample - loss: 0.0106 - accuracy: 0.9972
Epoch 4/10
6100/6100 [==============================] - 3s 495us/sample - loss: 0.0212 - accuracy: 0.9934
Epoch 5/10
6100/6100 [==============================] - 3s 522us/sample - loss: 0.0423 - accuracy: 0.9846
Epoch 6/10
6100/6100 [==============================] - 3s 509us/sample - loss: 0.0069 - accuracy: 0.9989
Epoch 7/10
6100/6100 [==============================] - 3s 491us/sample - loss: 0.0124 - accuracy: 0.9952
Epoch 8/10
6100/6100 [==============================] - 3s 513us/sample - loss: 0.0608 - accuracy: 0.9790
Epoch 9/10
6100/6100 [==============================] - 3s 536us/sample - loss: 0.0583 - accuracy: 0.9800
Epoch 10/10
6100/6100 [=

In [80]:
# print("Running prediction on training set")
# prediction_training_dir = "predictions_training/"
# if not os.path.isdir(prediction_training_dir):
#     os.mkdir(prediction_training_dir)
# for i in range(1, TRAINING_SIZE + 1):
#     pimg = get_prediction_with_groundtruth(model, train_data_filename, i)
#     Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
#     oimg = get_prediction_with_overlay(model, train_data_filename, i)
#     oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")